## STEP 1 Install Darknet


In [ ]:
%cd /content
!rm -r darknet/
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
import re
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make
!chmod +x ./darknet


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
#showing images
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  


In [ ]:
import numpy as np
import glob, os
import random
import numpy as np
import argparse
import time
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Data Preparation

In [ ]:
#Download data from the github source
!git clone https://github.com/thsant/wgisd

Cloning into 'wgisd'...
remote: Enumerating objects: 4248, done.
remote: Counting objects: 100% (848/848), done.
remote: Compressing objects: 100% (839/839), done.
remote: Total 4248 (delta 19), reused 820 (delta 7), pack-reused 3400
Receiving objects: 100% (4248/4248), 1.39 GiB | 14.28 MiB/s, done.
Resolving deltas: 100% (342/342), done.
Checking out files: 100% (1849/1849), done.


In [ ]:
L=[]
for filename in glob.glob('/content/darknet/wgisd/data/CFR_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r') as f:
       text = f.readlines()
       for i in range(len(text) ):
         text[i]='1'+str(text[i][1:])
       L.append(text)
for filename in glob.glob('/content/darknet/wgisd/data/CFR_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r+') as f:
     f.truncate(0)
     i=0
for filename in glob.glob('/content/darknet/wgisd/data/CFR_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'a') as f:
     f.truncate(0)
     f.writelines(L[i])
     i+=1

In [ ]:
L=[]
for filename in glob.glob('/content/darknet/wgisd/data/CSV_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r') as f:
       text = f.readlines()
       for i in range(len(text) ):
         text[i]='2'+str(text[i][1:])
       L.append(text)
for filename in glob.glob('/content/darknet/wgisd/data/CSV_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r+') as f:
     f.truncate(0)
     i=0
for filename in glob.glob('/content/darknet/wgisd/data/CSV_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'a') as f:
     f.truncate(0)
     f.writelines(L[i])
     i+=1

In [ ]:
L=[]
for filename in glob.glob('/content/darknet/wgisd/data/SVB_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r') as f:
       text = f.readlines()
       for i in range(len(text) ):
         text[i]='3'+str(text[i][1:])
       L.append(text)
for filename in glob.glob('/content/darknet/wgisd/data/SVB_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r+') as f:
     f.truncate(0)
     i=0
for filename in glob.glob('/content/darknet/wgisd/data/SVB_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'a') as f:
     f.truncate(0)
     f.writelines(L[i])
     i+=1

In [ ]:
L=[]
for filename in glob.glob('/content/darknet/wgisd/data/SYH_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r') as f:
       text = f.readlines()
       for i in range(len(text) ):
         text[i]='4'+str(text[i][1:])
       L.append(text)
for filename in glob.glob('/content/darknet/wgisd/data/SYH_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'r+') as f:
     f.truncate(0)
     i=0
for filename in glob.glob('/content/darknet/wgisd/data/SYH_*.txt'):
   with open(os.path.join(os.getcwd(), filename), 'a') as f:
     f.truncate(0)
     f.writelines(L[i])
     i+=1

In [ ]:
n_vimages = {v: len(inst_v) for v, inst_v in instances.items()}
n_vimages

# Transfer Learning

In [ ]:
repo_url = 'https://github.com/GotG/yolotinyv3_medmask_demo'
import os
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
%rm -r {repo_dir_path}
!git clone {repo_url}
%cd {repo_dir_path}


/content
rm: cannot remove '/content/yolotinyv3_medmask_demo': No such file or directory
Cloning into 'yolotinyv3_medmask_demo'...
remote: Enumerating objects: 1733, done.
remote: Total 1733 (delta 0), reused 0 (delta 0), pack-reused 1733
Receiving objects: 100% (1733/1733), 208.88 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (120/120), done.
Checking out files: 100% (1537/1537), done.
/content/yolotinyv3_medmask_demo


## Let's start with the images and annotations
We copy the images on our google drive in a folder. My gdrive folder is named medmask_yolo. Then we mount the gdrive in colab and copy the images and annotations in the obj folder. Change the **`if (False):`** to **`if (True):`** to use your own images from gdrive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## REMOVE THIS TO USE YOUR OWN IMAGES
if (True):
  #remove the obj folder
  !rm -rf /content/yolotinyv3_medmask_demo/obj
  #copy images to obj folder
  #this can take a few minutes depending on dataset size. 
  !cp -rf '/content/darknet/wgisd/data' '/content/yolotinyv3_medmask_demo/obj' 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Change the labels in obj.names to our current labels

In [ ]:
labels_path = '/content/yolotinyv3_medmask_demo/obj.names'
#make a list of your labels
labels = ['CDY', 'CFR', 'CSV', 'SVB', 'SYH']


with open(labels_path, 'w') as f:

    f.write('\n'.join(labels))

#check that the labels file is correct
!cat /content/yolotinyv3_medmask_demo/obj.names

CDY
CFR
CSV
SVB
SYH

## Change the number of classes in obj.data.
The paths are relative so no change there as long as the folder/file structure/names are not changed.

In [ ]:
import re
objdata = '/content/yolotinyv3_medmask_demo/obj.data'
with open(objdata) as f:
    s = f.read()

#the number of classes is equal to the number of labels
num_classes = len(labels)   
s = re.sub('classes = \d*','classes = ' + str(num_classes),s)

with open(objdata, 'w') as f:
  f.write(s)
!cat /content/yolotinyv3_medmask_demo/obj.data

classes= 5
train  = /content/yolotinyv3_medmask_demo/train.txt
valid  = /content/yolotinyv3_medmask_demo/valid.txt
names = /content/yolotinyv3_medmask_demo/obj.names
backup = backup/

In [ ]:
num_classes

5

## Now we need to create a train.txt, valid.txt and test.txt file for our images from `obj` folder.
We will use a script that splits the images based on a specified percentage 
and writes them in train.txt, valid.txt and test.txt files.

In [ ]:
%cd ../yolotinyv3_medmask_demo/

/content/yolotinyv3_medmask_demo


In [ ]:
#in this case we use 85 percent of the images for training
#10 percent for validation, 5 for testing.
#as the dataset is small, one can use 90 percent for training and split the rest
#for validation/testing as desired.
#double clicking on the train or test text files will open the file in a new
#tab in colab and display the content.
!python3 folder2textYolo.py 85 10 /content/yolotinyv3_medmask_demo/obj

Your image file extension is: .jpg
Number of images: 300
Number of images used for training 255
Number of images used for validation 30
Number of images used for testing 15


## Last thing we need to change is the yolov3-tiny_obj.cfg file.
There are several parameters of importance here. They control various aspects of the training process. Let's print the first few lines and have a look


In [ ]:
!head -n 24 /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg 


[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1



### Now that we see which parameters are of importance, let's modify them according to our dataset.

In [ ]:
# set the number of max_batches - min 2000 per class:
max_batch=4000
# calculate the 2 steps values:
step1 = 0.8 * max_batch
step2 = 0.9 * max_batch

# we also need to adjust the number of classes and a parameter called filter size 
# that are both is inside the model structure

# num_classes = len(labels)
num_filters = (num_classes + 5) * 3


cfg_file = '/content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg'
# cfg_file = '/content/yolotinyv3_medmask_demo/yolov4-tiny.cfg'

with open(cfg_file) as f:
    s = f.read()
# (re.sub('[a-z]*@', 'ABC@', s))
s = re.sub('max_batches = \d*','max_batches = '+str(max_batch),s)
s = re.sub('steps=\d*,\d*','steps='+"{:.0f}".format(step1)+','+"{:.0f}".format(step2),s)
s = re.sub('classes=\d*','classes='+str(num_classes),s)
s = re.sub('pad=1\nfilters=\d*','pad=1\nfilters='+"{:.0f}".format(num_filters),s)
# pad=1\nfilters=\d\d
# s = re.sub('CUDNN=0','CUDNN=1',s)
# s = re.sub('OPENCV=0','OPENCV=1',s)

with open(cfg_file, 'w') as f:
  # s = re.sub('GPU=0','GPU=1',s)
  f.write(s)



Let's look again at the configuration 

In [ ]:
!head -n 24 /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg 


[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=3200,3600
scales=.1,.1



We also check if the number of classes and filters updated correctly

In [ ]:
!tail -n 64 /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg 


stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=30
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=5
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 8

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=30
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=5
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1


# Train the model

When you execute the following command, your model will start training.

You will have a log line per epoch. On each iteration you will see how your training is going.

In the `content/darknet/backup/` folder Darknet saves a few weights files:
* files that end in 1000,2000 etc are weights saved every 1000 batches
* best weights file are the weights that gave best results during training
* final weights file contains the final weights at the end of the training





## Start the model training


In [ ]:
%cd /content/darknet/

/content/darknet


In [ ]:
!./darknet detector train /content/yolotinyv3_medmask_demo/obj.data /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg /content/yolotinyv3_medmask_demo/yolov3-tiny.conv.15 -dont_show -ext_output -map

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.750199), count: 2, class_loss = 0.200726, iou_loss = 0.061800, total_loss = 0.262527 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.613324), count: 50, class_loss = 7.082613, iou_loss = 6.623942, total_loss = 13.706554 
 total_bbox = 1211572, rewritten_bbox = 0.175887 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.000000), count: 1, class_loss = 0.071193, iou_loss = 0.000000, total_loss = 0.071193 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.612108), count: 40, class_loss = 7.690645, iou_loss = 5.410954, total_loss = 13.101600 
 total_bbox = 1211612, rewritten_bbox = 0.175881 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.656846), count: 5, class_loss = 0.984905, iou_loss = 0.50073

## Check the model performance
The metrics run on the validation images so they may not be fully representative, only indicative

In [ ]:
!./darknet detector map /content/yolotinyv3_medmask_demo/obj.data /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg "/content/darknet/backup/yolov3-tiny_obj_best.weights" -points 0

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     16       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  16 0.150 BF
   1 max                2x 2/ 2    416 x 416 x  16 ->  208 x 208 x  16 0.003 BF
   2 conv     32       3 x 3/ 1    208 x 208 x  16 ->  208 x 208 x  32 0.399 BF
   3 max                2x 2/ 2    208 x 208 x  32 ->  104 x 104 x  32 0.001 BF
   4 conv     64       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  64 0.399 BF
   5 max                2x 2/ 2    104 x 104 x  64 ->   52 x  52 x  64 0.001 BF
   6 conv    128       3 x 3/ 1     52 x  52 x  64 ->   52 x  52 x 128 0.399 BF
   7 max                2x 2/ 2     52 x  52 x 128 ->   26 x  26 x 128 0.000 BF
   8 conv    256  

## Run detection on an image. I choose one of the test images from test.txt

In [ ]:
!./darknet detector test /content/yolotinyv3_medmask_demo/obj.data  /content/yolotinyv3_medmask_demo/yolov3-tiny_obj.cfg  "/content/darknet/backup/yolov3-tiny_obj_best.weights" /content/yolotinyv3_medmask_demo/obj/0633.jpg -ext_output
imShow('predictions.jpg')

/bin/bash: ./darknet: No such file or directory


NameError: ignored

## Copy best weights to google drive


In [ ]:
!cp /content/darknet/backup/yolov3-tiny_obj_best.weights  '/content/drive/My Drive/'